# Appliying the simplest image classification algorithm: k-Nearest Neighbor (k-NN) on MNIST dataset.



This algorithm is so simple that it doesn’t do any actual “learning” — yet it is still heavily used in many computer vision algorithms. And as we’ll see, we’ll be able to use this algorithm to recognize handwritten digits from the our  MNIST dataset.

To do that we will go threw the following objectives:


- Have an understanding of the k-Nearest Neighbor classifier.
- Know how to apply the k-Nearest Neighbor classifier to image datasets.
- Be able to recognize handwritten digits from (a sample of) the MNIST dataset.


### The k-Nearest Neighbor Classifier

The k-Nearest Neighbor classifier is by far the most simple image classification algorithm. In fact, it’s so simple that it doesn’t actually “learn” anything! Instead, this algorithm simply relies on the distance between feature vectors, much like in building an image search engine — only this time, we have the labels associated with each image, so we can predict and return an actual category for the image.

Simply put, the k-NN algorithm classifies unknown data points by finding the most common class among the k closest examples. Each data point in the k closest data points casts a vote, and the category with the highest number of votes wins! Or in plain english: “Tell me who your neighbors are, and I’ll tell you who you are”.

<img src="images/knn_class.jpg" width='480px' style="float:left">
<img src="images/knn.jpg" width='480px' style="float:right">

n order for the k-NN algorithm to work, it makes the primary assumption that feature vectors that lie close together in an n-dimensional space have similar visual contents

In the image on the right, we can see three categories of images, denoted as red, blue, and green dots, respectively. We can see that each of these sets of data points are grouped relatively close together in our n-dimensional space. This implies that the distance between two red dots is much smaller than the distance between a red dot and a blue dot.

However, in order to apply the k-Nearest Neighbor classifier, we first need to select a distance metric or a similarity function. We briefly discussed the Euclidean distance (often called the L2-distance) in our lesson on color channel statistics:

### k-NN in action

At this point, we showed the principles of the k-NN algorithm. We know that it relies on the distance between feature vectors to make a classification. And we know that it requires a distance metric/similarity function to compute these distances.

But how do we actually make the classification?

To answer this question, let’s look at the following figure:

<img src="images/1.jpg" width='300px' style="float:left">
<img src="images/2.jpg" width='300px' style="float:left">
<img src="images/3.jpg" width='300px' style="float:left">

Here, we have a dataset of three types of flowers — sunflowers, daises, and pansies — and we have plotted them according to the size and lightness of their petals.

Now, let’s insert a new, unknown flower and try to classify it using only a single neighbor (i.e. k=1):

Here, we have found the “nearest neighbor” to our test flower, indicated by k=1. And according to the label of the nearest flower, it’s a daisy.

Let’s try another “unknown flower”, this time using k=3

This time, we have found two sunflowers and one daisy in the top three results. Since the sunflower category has the largest number of votes, we’ll classify this unknown flower as a sunflower.

We can keep performing this process for varying values of k, but no matter how large or small k becomes, the principle remains the same — the category with the largest number of votes in the k closest training points wins and is used as the label for the testing point.

###                                Recognizing handwritten digits using MNIST

<img src="images/mnist.jpg" style="float:left">

The MNIST dataset is one of the most well studied datasets in the computer vision and machine learning literature. In many cases, it’s a benchmark and a standard to which machine learning algorithms are ranked.

The goal of this dataset is to correctly classify the handwritten digits 0-9. Instead of utilizing the entire dataset (which consists of 60,000 training images and 10,000 testing images,) we’ll be using a small subset of the data provided by the scikit-learn library — this subset includes 1,797 digits, which we’ll split into training, validation, and testing sets, respectively.



__Our goal here is to train a k-NN classifier on the raw pixel intensities and then classify unknown digits.__

To accomplish this goal, we’ll be using our five-step pipeline to train image classifiers:


   - Step 1 – Structuring our initial dataset: Our initial dataset consists of 1,797 digits representing the numbers 0-9. These images are grayscale, 8 x 8 images with digits appearing as white on a black background. These digits have also been heavily pre-processed, aligned, and centered, making our classification job slightly easier.
   



In [ ]:
import numpy as np
import tensorflow as tf

# Import MNIST data

import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)


- Step 2 – Splitting the dataset: We’ll be using three splits for our experiment. The first set is our training set, used to train our k-NN classifier. And we’ll finally evaluate our classifier using the testing set.

In [2]:
# In this example, we limit mnist data
Xtr, Ytr = mnist.train.next_batch(60000)  # whole training set
Xtv, Ytv = mnist.validation.next_batch(1000)
Xte, Yte = mnist.test.next_batch(1000)
# 
xtr = tf.placeholder("float", [None, 784])
ytr = tf.placeholder("float", [None, 10])
xte = tf.placeholder("float", [784])

 - Step 3 – 
 Calculation of the distance between training and testing images and return the min distance neighboors, test shows thaht the best k is 4.

In [3]:
K=10
# Euclidean Distance
distance = tf.negative(tf.sqrt(tf.reduce_sum(tf.square(tf.subtract(xtr, xte)), reduction_indices=1)))
# Prediction: Get min distance neighbors
values, indices = tf.nn.top_k(distance, k=K, sorted=False)

   - Step 4 – Training our classification model: Our k-NN classifier will be trained on the raw pixel intensities of the images in the training set.

In [4]:

nearest_neighbors = []
for i in range(K):
    nearest_neighbors.append(tf.argmax(ytr[indices[i]], 0))

neighbors_tensor = tf.stack(nearest_neighbors)
y, idx, count = tf.unique_with_counts(neighbors_tensor)
pred = tf.slice(y, begin=[tf.argmax(count, 0)], size=tf.constant([1], dtype=tf.int64))[0]

accuracy = 0.


  

   - Step 5 – Evaluating our classifier: Once we have found the best value of k, we can then evaluate our k-NN classifier on our testing set.

In [5]:
# Initializing the variables
init = tf.initialize_all_variables()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    # loop over test data
    for i in range(len(Xte)):
        # Get nearest neighbor
        nn_index = sess.run(pred, feed_dict={xtr: Xtr, ytr: Ytr, xte: Xte[i, :]})
        # Get nearest neighbor class label and compare it to its true label
        if((i%100)==0 or (i%1000)==0): 
            print("Test", i, "Prediction:", nn_index,"True Class:", np.argmax(Yte[i]))
        #Calculate accuracyfg
        if nn_index == np.argmax(Yte[i]):
            accuracy += 1. / len(Xte)
    print("Done!")
    print("Accuracy:", accuracy)

Instructions for updating:
Use `tf.global_variables_initializer` instead.
('Test', 0, 'Prediction:', 1, 'True Class:', 1)
('Test', 100, 'Prediction:', 7, 'True Class:', 7)
('Test', 200, 'Prediction:', 6, 'True Class:', 6)
('Test', 300, 'Prediction:', 3, 'True Class:', 3)
('Test', 400, 'Prediction:', 0, 'True Class:', 0)
('Test', 500, 'Prediction:', 3, 'True Class:', 3)
('Test', 600, 'Prediction:', 0, 'True Class:', 0)
('Test', 700, 'Prediction:', 9, 'True Class:', 9)
('Test', 800, 'Prediction:', 9, 'True Class:', 9)
('Test', 900, 'Prediction:', 7, 'True Class:', 7)
Done!
('Accuracy:', 0.9700000000000008)
